# FreshHarvest Deployment Testing

This notebook provides comprehensive testing for FreshHarvest model deployment including:
- Model loading and inference testing
- Performance benchmarking
- API endpoint testing
- Streamlit application testing
- Docker deployment validation
- Production readiness checks

In [1]:
import os
import sys
import time
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Import custom modules
from cvProject_FreshHarvest.utils.common import read_yaml, setup_logging
from cvProject_FreshHarvest.models.cnn_models import FreshHarvestCNN

# Setup
setup_logging()
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print(f"Python version: {sys.version}")

TensorFlow version: 2.19.0
GPU available: []
Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


## 1. Model Loading Tests

In [2]:
# Test model loading from different sources
def test_model_loading():
    """Test loading models from different file formats and locations."""
    
    print("\n🧪 Testing Model Loading...")
    print("=" * 50)
    
    # Test different model paths
    model_paths = [
        '../models/trained/best_model.h5',
        '../models/best_hypertuned_model.h5',
        '../models/checkpoints/best_model_20250618_100126.h5'
    ]
    
    results = []
    
    for path in model_paths:
        result = {
            'path': path,
            'exists': os.path.exists(path),
            'loadable': False,
            'load_time': None,
            'model_size': None,
            'parameters': None
        }
        
        if result['exists']:
            try:
                start_time = time.time()
                model = keras.models.load_model(path)
                load_time = time.time() - start_time
                
                result['loadable'] = True
                result['load_time'] = load_time
                result['model_size'] = os.path.getsize(path) / (1024 * 1024)  # MB
                result['parameters'] = model.count_params()
                
                print(f"✅ {path}")
                print(f"   Load time: {load_time:.3f}s")
                print(f"   Model size: {result['model_size']:.2f} MB")
                print(f"   Parameters: {result['parameters']:,}")
                
            except Exception as e:
                print(f"❌ {path}: {e}")
        else:
            print(f"⚠️ {path}: File not found")
        
        results.append(result)
    
    return results

# Run model loading tests
loading_results = test_model_loading()


🧪 Testing Model Loading...
⚠️ ../models/trained/best_model.h5: File not found
⚠️ ../models/best_hypertuned_model.h5: File not found
2025-06-19 07:54:13,775 - absl - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
✅ ../models/checkpoints/best_model_20250618_100126.h5
   Load time: 0.411s
   Model size: 1.04 MB
   Parameters: 83,019


## 2. Inference Performance Testing

In [3]:
# Test inference performance
def test_inference_performance(model, num_tests=100):
    """Test model inference performance."""
    
    print(f"\n⚡ Testing Inference Performance ({num_tests} iterations)...")
    print("=" * 60)
    
    # Create dummy test data
    test_images = np.random.random((num_tests, 224, 224, 3))
    
    # Warm up the model
    _ = model.predict(test_images[:5], verbose=0)
    
    # Test single image inference
    single_times = []
    for i in range(min(20, num_tests)):
        start_time = time.time()
        _ = model.predict(test_images[i:i+1], verbose=0)
        inference_time = time.time() - start_time
        single_times.append(inference_time)
    
    # Test batch inference
    batch_sizes = [1, 8, 16, 32]
    batch_results = []
    
    for batch_size in batch_sizes:
        if batch_size <= num_tests:
            start_time = time.time()
            _ = model.predict(test_images[:batch_size], verbose=0)
            total_time = time.time() - start_time
            avg_time_per_image = total_time / batch_size
            
            batch_results.append({
                'batch_size': batch_size,
                'total_time': total_time,
                'avg_time_per_image': avg_time_per_image,
                'throughput': batch_size / total_time
            })
    
    # Display results
    print(f"Single Image Inference:")
    print(f"  Mean time: {np.mean(single_times):.4f}s")
    print(f"  Std time: {np.std(single_times):.4f}s")
    print(f"  Min time: {np.min(single_times):.4f}s")
    print(f"  Max time: {np.max(single_times):.4f}s")
    
    print(f"\nBatch Inference Results:")
    for result in batch_results:
        print(f"  Batch size {result['batch_size']:2d}: {result['avg_time_per_image']:.4f}s/image, {result['throughput']:.2f} images/s")
    
    return single_times, batch_results

# Load a model for testing
test_model = None
for result in loading_results:
    if result['loadable']:
        test_model = keras.models.load_model(result['path'])
        print(f"\nUsing model: {result['path']}")
        break

if test_model is None:
    print("\n⚠️ No trained model available. Creating lightweight model for testing.")
    config = read_yaml('../config/config.yaml')
    cnn_builder = FreshHarvestCNN('../config/config.yaml')
    test_model = cnn_builder.create_lightweight_cnn()
    test_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Run performance tests
single_times, batch_results = test_inference_performance(test_model)

2025-06-19 07:54:14,311 - absl - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.

Using model: ../models/checkpoints/best_model_20250618_100126.h5

⚡ Testing Inference Performance (100 iterations)...
Single Image Inference:
  Mean time: 0.1830s
  Std time: 0.0619s
  Min time: 0.1274s
  Max time: 0.4213s

Batch Inference Results:
  Batch size  1: 0.2033s/image, 4.92 images/s
  Batch size  8: 0.0297s/image, 33.69 images/s
  Batch size 16: 0.0229s/image, 43.76 images/s
  Batch size 32: 0.0132s/image, 76.03 images/s


## 3. Memory Usage Testing

In [4]:
# Test memory usage
def test_memory_usage():
    """Test memory usage during model operations."""
    
    print("\n💾 Testing Memory Usage...")
    print("=" * 40)
    
    try:
        import psutil
        process = psutil.Process()
        
        # Initial memory
        initial_memory = process.memory_info().rss / (1024 * 1024)  # MB
        print(f"Initial memory usage: {initial_memory:.2f} MB")
        
        # Memory after model loading
        if test_model is not None:
            model_memory = process.memory_info().rss / (1024 * 1024)  # MB
            print(f"Memory after model loading: {model_memory:.2f} MB")
            print(f"Model memory overhead: {model_memory - initial_memory:.2f} MB")
            
            # Memory during inference
            test_batch = np.random.random((32, 224, 224, 3))
            _ = test_model.predict(test_batch, verbose=0)
            inference_memory = process.memory_info().rss / (1024 * 1024)  # MB
            print(f"Memory during inference: {inference_memory:.2f} MB")
            print(f"Inference memory overhead: {inference_memory - model_memory:.2f} MB")
            
            # Memory efficiency
            model_params = test_model.count_params()
            theoretical_size = model_params * 4 / (1024 * 1024)  # Assuming float32
            print(f"\nModel efficiency:")
            print(f"  Theoretical model size: {theoretical_size:.2f} MB")
            print(f"  Actual memory usage: {model_memory - initial_memory:.2f} MB")
            print(f"  Memory efficiency: {theoretical_size / (model_memory - initial_memory) * 100:.1f}%")
        
    except ImportError:
        print("⚠️ psutil not available. Install with: pip install psutil")
    except Exception as e:
        print(f"❌ Memory testing failed: {e}")

# Run memory usage tests
test_memory_usage()


💾 Testing Memory Usage...
Initial memory usage: 869.22 MB
Memory after model loading: 869.23 MB
Model memory overhead: 0.01 MB
Memory during inference: 865.73 MB
Inference memory overhead: -3.50 MB

Model efficiency:
  Theoretical model size: 0.32 MB
  Actual memory usage: 0.01 MB
  Memory efficiency: 4053.7%


## 4. Streamlit Application Testing

In [5]:
# Test Streamlit application
def test_streamlit_app():
    """Test if Streamlit application can be started."""
    
    print("\n🌐 Testing Streamlit Application...")
    print("=" * 45)
    
    # Check if Streamlit files exist
    streamlit_files = [
        '../app_simple.py',
        '../app.py'
    ]
    
    for app_file in streamlit_files:
        if os.path.exists(app_file):
            print(f"✅ Found Streamlit app: {app_file}")
            
            # Check if the file is valid Python
            try:
                with open(app_file, 'r') as f:
                    content = f.read()
                    
                # Basic syntax check
                compile(content, app_file, 'exec')
                print(f"   ✅ Syntax check passed")
                
                # Check for required imports
                required_imports = ['streamlit', 'tensorflow', 'numpy']
                for imp in required_imports:
                    if imp in content:
                        print(f"   ✅ {imp} import found")
                    else:
                        print(f"   ⚠️ {imp} import not found")
                        
            except SyntaxError as e:
                print(f"   ❌ Syntax error: {e}")
            except Exception as e:
                print(f"   ❌ Error checking file: {e}")
        else:
            print(f"❌ Streamlit app not found: {app_file}")
    
    # Test if Streamlit can be imported
    try:
        import streamlit as st
        print(f"\n✅ Streamlit version: {st.__version__}")
    except ImportError:
        print(f"\n❌ Streamlit not installed. Install with: pip install streamlit")

# Run Streamlit tests
test_streamlit_app()


🌐 Testing Streamlit Application...
✅ Found Streamlit app: ../app_simple.py
   ❌ Error checking file: 'charmap' codec can't decode byte 0x8d in position 756: character maps to <undefined>
✅ Found Streamlit app: ../app.py
   ❌ Error checking file: 'charmap' codec can't decode byte 0x8d in position 937: character maps to <undefined>

✅ Streamlit version: 1.45.1
